In [38]:
import json
import os
import shutil
import cv2
import numpy as np
from imantics import Polygons, Mask

In [66]:
def copy_file(src: str, dst: str) -> None:
    os.makedirs(os.path.dirname(dst), exist_ok=True)
    if not os.path.exists(dst):
        shutil.copy(src , dst)

def gen_yolov7_txt(polygons:np.ndarray, cls:int, save_path: str):
    """
    Args:
        polygons: a array contain multiple line of [x1, y1, x2, y2, ... xn, yn]
        cls: the index of class
        save_path: A path for saving text file of yolov7 format 
    """
    lines = []
    for points in polygons:
        line = [str(cls)]
        line.extend([v for v in points])
        line = ' '.join(line) + '\n'
        lines.append(line)

    os.makedirs(os.path.dirname(save_path), exist_ok=True)
    with open(save_path, 'w') as f:
        f.writelines(lines)

def mask_to_polygons(mask_path: str) -> np.ndarray:
    im = cv2.imread(mask_path, 0)
    unique = np.unique(im)
    if len(unique) != 2:
        raise ValueError("Some Error in {}".format(mask_path))
    ret, im = cv2.threshold(im, unique[1] - 1, 255, cv2.THRESH_BINARY)
    h, w = im.shape
    polygons = Mask(im).polygons()

    normalized_points = []
    for points in polygons.points:
        points = points.astype(np.float32)
        points[..., 0] /= w # x
        points[..., 1] /= h # y
        normalized_points.append(points)

    # Reshape into multuple line of [x1, y1, x2, y2, ... xn, yn]
    return normalized_points


class_names = ['powder_uncover', 'powder_uneven', 'scratch']
for split_name in ['Train', 'Val']:
    data_root = os.path.join('./origin_data', split_name)
    new_data_root = os.path.join('./data', split_name)

    name_idx = 0
    for cls_idx, cls in enumerate(class_names):
        cls_root = os.path.join(data_root, cls)
        for name in os.listdir(os.path.join(cls_root, 'image')):
            # Rename and move image
            new_name = f'{name_idx}.png'
            copy_file(os.path.join(cls_root, 'image', name), 
                    os.path.join(new_data_root, 'images', new_name))
            mask_path = os.path.join(cls_root, 'mask', name)
            polygons = mask_to_polygons(mask_path)
            yolov7_txt_file_name = f"{name_idx}.txt"
            gen_yolov7_txt(polygons, cls_idx, os.path.join(new_data_root, 'labels', yolov7_txt_file_name))
            name_idx += 1

ValueError: all input arrays must have the same shape

array([[[0.46969697, 0.00160772],
        [0.46969697, 0.99678457],
        [0.48883572, 0.99678457],
        [0.48883572, 0.00160772]],

       [[0.40350878, 0.        ],
        [0.40350878, 0.9975884 ],
        [0.41786283, 0.9975884 ],
        [0.41786283, 0.        ]],

       [[0.33173844, 0.        ],
        [0.33173844, 0.9959807 ],
        [0.34688994, 0.9959807 ],
        [0.34688994, 0.        ]]], dtype=float32)

In [4]:
label_dir = Path('./data/Train/scratch/label/')
label_files = [label_dir / f for f in os.listdir(label_dir)]

In [8]:
with open(label_files[0], 'r') as f:
    data = json.load(f)

In [22]:


# This can be any array
img_path = './origin_data/Train/scratch/mask/Converted_ 0003.png'



In [37]:
points.reshape(3, -1)

array([[0.46969697, 0.00160772, 0.46969697, 0.99678457, 0.48883572,
        0.99678457, 0.48883572, 0.00160772],
       [0.40350878, 0.        , 0.40350878, 0.9975884 , 0.41786283,
        0.9975884 , 0.41786283, 0.        ],
       [0.33173844, 0.        , 0.33173844, 0.9959807 , 0.34688994,
        0.9959807 , 0.34688994, 0.        ]], dtype=float32)

In [36]:
points

array([[[0.46969697, 0.00160772],
        [0.46969697, 0.99678457],
        [0.48883572, 0.99678457],
        [0.48883572, 0.00160772]],

       [[0.40350878, 0.        ],
        [0.40350878, 0.9975884 ],
        [0.41786283, 0.9975884 ],
        [0.41786283, 0.        ]],

       [[0.33173844, 0.        ],
        [0.33173844, 0.9959807 ],
        [0.34688994, 0.9959807 ],
        [0.34688994, 0.        ]]], dtype=float32)

In [ ]:
np.flatt

In [32]:
np.array(polygons.points)

array([[[ 589,    2],
        [ 589, 1240],
        [ 613, 1240],
        [ 613,    2]],

       [[ 506,    0],
        [ 506, 1241],
        [ 524, 1241],
        [ 524,    0]],

       [[ 416,    0],
        [ 416, 1239],
        [ 435, 1239],
        [ 435,    0]]])

array([[[0.46969697, 0.00160772],
        [0.46969697, 0.99678457],
        [0.48883572, 0.99678457],
        [0.48883572, 0.00160772]],

       [[0.40350878, 0.        ],
        [0.40350878, 0.9975884 ],
        [0.41786283, 0.9975884 ],
        [0.41786283, 0.        ]],

       [[0.33173844, 0.        ],
        [0.33173844, 0.9959807 ],
        [0.34688994, 0.9959807 ],
        [0.34688994, 0.        ]]], dtype=float32)

[array([[ 589,    2],
        [ 589, 1240],
        [ 613, 1240],
        [ 613,    2]]),
 array([[ 506,    0],
        [ 506, 1241],
        [ 524, 1241],
        [ 524,    0]]),
 array([[ 416,    0],
        [ 416, 1239],
        [ 435, 1239],
        [ 435,    0]])]